# Libraries

In [1]:
import pandas as pd
import itertools

# Read Data

In [2]:
df = pd.read_excel("../data/example_3dim.xlsx")
df.head(6)

,Xone,Yone,Zone,Value
0,1,1,3,98.320407
1,1,2,3,6.869262
2,1,3,3,0.464133
3,1,4,3,2.705966
4,2,1,3,70.177818
5,2,3,3,44.725465


# Derivatives

In [3]:
# Basic Definitions
featuresArray = list(df.columns)[:-1]
TargetArray = list(df.columns)[-1]
# Get Range of Data
featuresMaxArray = df[featuresArray].max()
featuresMinArray = df[featuresArray].min()
#Complete Range
featuresRangeArray = []
for featuresRangeArrayIte in range(len(featuresMaxArray)):
    featuresRangeArray.append(range(featuresMinArray[featuresRangeArrayIte],featuresMaxArray[featuresRangeArrayIte]+1))

In [4]:
#Get Range Array
allRangeArray = []
for i in featuresRangeArray:
    allRangeArray.append(list(i))
#Create Data Frame Without Empty Spaces
dfT = pd.DataFrame(itertools.product(*allRangeArray),columns=featuresArray)
dfT = pd.merge(left=dfT,right=df,on=featuresArray,how='left')
dfT.head(6)

,Xone,Yone,Zone,Value
0,1,1,3,98.320407
1,1,1,4,27.508250
2,1,2,3,6.869262
3,1,2,4,16.624032
4,1,3,3,0.464133
5,1,3,4,51.718670


In [5]:
def catchByFilter(df,fil):
    #Query
    qry = ' and '.join(["{} == {}".format(k,v) for k,v in fil.items()])   
    #Return
    return(df.query(qry).reset_index())

In [6]:
step = 1
listDictsGradient = []
for gradientFeature in featuresArray:
    
    #Cada Linha do DF
    print("Gradient "+gradientFeature+" --> "+"G_"+gradientFeature)
    for eachRow in range(len(dfT)):
        
        #Filters
        ##Point
        filterPoint = dfT.iloc[eachRow,:][featuresArray]
        ##Before:
        filterBefore = filterPoint.copy()
        filterBefore[gradientFeature] -= step
        #Filtros
        filterAfter = filterPoint.copy()
        filterAfter[gradientFeature] += step
        
        
        #Catch Data
        dfP = catchByFilter(df=dfT,fil=filterPoint)
        dfB = catchByFilter(df=dfT,fil=filterBefore)
        dfA = catchByFilter(df=dfT,fil=filterAfter)
        
        #Denominator
        if (len(dfB)+len(dfP)+len(dfA)) == 3:
            denominator = 2
        else:
            denominator = 1
        
        #Gradient
        if len(dfB) == 0:
            gradient = (dfA[TargetArray] - dfP[TargetArray])/denominator
        elif len(dfA) == 0:
            gradient = (dfP[TargetArray] - dfB[TargetArray])/denominator
        elif (len(dfB)+len(dfA)) == 2:
            gradient = (dfA[TargetArray] - dfB[TargetArray])/denominator
            
        #Insert Gradient
        outDict = dfP.to_dict(orient='records')[0]
        ##Remove Index
        del outDict['index'] 
        ##Add Gradient
        outDict[('G_'+gradientFeature)]=gradient[0]
        ##Apend
        listDictsGradient.append(outDict)
       
#Join All Data
dfDict = pd.DataFrame(listDictsGradient)
dfDict

Gradient Xone --> G_Xone
Gradient Yone --> G_Yone
Gradient Zone --> G_Zone


,Xone,Yone,Zone,Value,G_Xone,G_Yone,G_Zone
0,1,1,3,98.320407,-28.142589,NaN,NaN
1,1,1,4,27.508250,43.094503,NaN,NaN
2,1,2,3,6.869262,NaN,NaN,NaN
3,1,2,4,16.624032,NaN,NaN,NaN
4,1,3,3,0.464133,44.261331,NaN,NaN
...,...,...,...,...,...,...,...
67,3,2,4,78.482144,NaN,NaN,-7.540639
68,3,3,3,53.788543,NaN,NaN,-53.558898
69,3,3,4,0.229645,NaN,NaN,-53.558898
70,3,4,3,32.430343,NaN,NaN,20.366547


In [7]:
#Group All Data
dfTF = dfT.copy()
for iGradient in featuresArray:
    filterGradient = ("G_"+iGradient)
    dfDictFilterGradient = dfDict.loc[~dfDict[filterGradient].isnull(),(featuresArray+[TargetArray,filterGradient])]
    dfTF = pd.merge(left=dfTF,right=dfDictFilterGradient,how='left',on=featuresArray+[TargetArray])
dfTF

,Xone,Yone,Zone,Value,G_Xone,G_Yone,G_Zone
0,1,1,3,98.320407,-28.142589,-91.451145,-70.812157
1,1,1,4,27.508250,43.094503,-10.884218,-70.812157
2,1,2,3,6.869262,NaN,-48.928137,9.754770
3,1,2,4,16.624032,NaN,12.105210,9.754770
4,1,3,3,0.464133,44.261331,-2.081648,51.254537
5,1,3,4,51.718670,-36.482546,14.855174,51.254537
6,1,4,3,2.705966,95.618413,2.241833,43.628414
7,1,4,4,46.334381,-43.038526,-5.384290,43.628414
8,2,1,3,70.177818,-20.859888,NaN,0.424936
9,2,1,4,70.602754,35.185051,NaN,0.424936
